
## References
1. Biopython handbook for more pdb parsing functions: https://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ
2. Deep Learning. structure based solubility prediction: https://jcheminf.biomedcentral.com/articles/10.1186/s13321-021-00488-1
3. Features used by another ML based prediction paper: https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-15-134/tables/3
4. SolArt github: https://github.com/minghuilab/PremPS/blob/e7cf74a467677bad50e20161777ca609362de1e3/v1.0.0/PremPS.py#L304
5. SOLart paper: https://academic.oup.com/bioinformatics/article/36/5/1445/5585748
6. https://github.com/biopython/biopython/blob/master/Bio/SeqUtils/ProtParamData.py#L6


In [156]:
!pip install --user freesasa
!pip install --user DSSPparser

In [174]:
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import PPBuilder
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import csv
import freesasa
import os
from os.path import join as join_p
import predict

ROOT_DIR = os.getcwd()
hydrophobic = ['V', 'I', 'L', 'F', 'M', 'W', 'Y', 'C']
ambigous_aa = ['B','X','J','Z']
negatively_charged = ['D', 'E']
positively_charged = ['R', 'K']
normal_format_pro = ['CYS','GLN','ILE','SER','VAL','MET','ASN','PRO','LYS','THR','PHE','ALA','HIS','GLY','ASP','LEU',
                     'ARG','TRP','GLU','TYR']

# map residue name three letters to one
map_three_one = {"GLY": "G", "ALA": "A", "SER": "S", "THR": "T", "CYS": "C",
                 "VAL": "V", "LEU": "L", "ILE": "I", "MET": "M", "PRO": "P",
                 "PHE": "F", "TYR": "Y", "TRP": "W", "ASP": "D", "GLU": "E",
                 "ASN": "N", "GLN": "Q", "HIS": "H", "LYS": "K", "ARG": "R",
                 "ASX": "X", "GLX": "X", "CSO": "X", "HIP": "X", "MSE": "X",
                 "UNK": "X", "SEC": "X", "PYL": "X", "SEP": "X", "TPO": "X",
                 "PTR": "X", "XLE": "X", "XAA": "X", "HSD": "H", "HID": "H",
                 "HSE": "H"}

# map residue name one letter to three
map_one_three = {"G": "GLY", "A": "ALA", "S": "SER", "T": "THR", "C": "CYS",
                 "V": "VAL", "L": "LEU", "I": "ILE", "M": "MET", "P": "PRO",
                 "F": "PHE", "Y": "TYR", "W": "TRP", "D": "ASP", "E": "GLU",
                 "N": "ASN", "Q": "GLN", "H": "HIS", "K": "LYS", "R": "ARG"}

# SASA_sol
map_surface = {'A':118.1,'R':256.0,'N':165.5,'D':158.7,'C':146.1,'Q':193.2,
               'E':186.2,'G':88.1,'H':202.5,'I':181.0,'L':193.1,'K':225.8,
               'M':203.4,'F':222.8,'P':146.8,'S':129.8,'T':152.5,'W':266.3,
               'Y':236.8,'V':164.5,'X':88.1}

DSSP_codes = {"H" # = α-helix
              "B", #= residue in isolated β-bridge
              "E",  # = extended strand, participates in β ladder
              "G",  # = 3-helix(310 helix)
              "I",  # = 5 helix(π-helix)
              "T",  # = hydrogen bonded turn
              "S",  # = bend 
             }

# Kyte & Doolittle index of hydrophobicity
# J. Mol. Biol. 157:105-132(1982).
kd = {"A": 1.8, "R": -4.5, "N": -3.5, "D": -3.5, "C": 2.5,
      "Q": -3.5, "E": -3.5, "G": -0.4, "H": -3.2, "I": 4.5,
      "L": 3.8, "K": -3.9, "M": 1.9, "F": 2.8, "P": -1.6,
      "S": -0.8, "T": -0.7, "W": -0.9, "Y": -1.3, "V": 4.2}

# Hydrophilicity
# 1 Hopp & Wood
# Proc. Natl. Acad. Sci. U.S.A. 78:3824-3828(1981).
hw = {"A": -0.5, "R": 3.0, "N": 0.2, "D": 3.0, "C": -1.0,
      "Q": 0.2, "E": 3.0, "G": 0.0, "H": -0.5, "I": -1.8,
      "L": -1.8, "K": 3.0, "M": -1.3, "F": -2.5, "P": 0.0,
      "S": 0.3, "T": -0.4, "W": -3.4, "Y": -2.3, "V": -1.5}

# Surface accessibility
# Vergoten G & Theophanides T, Biomolecular Structure and Dynamics,
# pg.138 (1997).
# 1 Emini Surface fractional probability
em = {"A": 0.815, "R": 1.475, "N": 1.296, "D": 1.283, "C": 0.394,
      "Q": 1.348, "E": 1.445, "G": 0.714, "H": 1.180, "I": 0.603,
      "L": 0.603, "K": 1.545, "M": 0.714, "F": 0.695, "P": 1.236,
      "S": 1.115, "T": 1.184, "W": 0.808, "Y": 1.089, "V": 0.606}

# 2 Janin Interior to surface transfer energy scale
ja = {"A": 0.28, "R": -1.14, "N": -0.55, "D": -0.52, "C": 0.97,
      "Q": -0.69, "E": -1.01, "G": 0.43, "H": -0.31, "I": 0.60,
      "L": 0.60, "K": -1.62, "M": 0.43, "F": 0.46, "P": -0.42,
      "S": -0.19, "T": -0.32, "W": 0.29, "Y": -0.15, "V": 0.60}

global pdb_list, pdb_matrix, solubility_map
pdb_list = []
pdb_matrix = []
solubility_map = {}

def generate_pdb_list ():
    with open ("pdb_list.txt", "r") as file:
        for line in file:
            pdb_list.append(line.strip())

def generate_solubility_mapping ():
    with open("data/training/crystal_structs/solubility_values.csv", "r") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)

        for row in csvreader:
            pdb_name = row[0].strip()
            solubility_score = int(row[1].strip())
            solubility_map[pdb_name] = solubility_score
            
def pdb_data_curation():
    if pdb_list:
        for pdb in pdb_list:
            pdb_path = join_p(ROOT_DIR,"data/training/crystal_structs/" + pdb)        
            pdb_name = pdb.rstrip(".pdb")   
            
            #Extracting sequence
            structure = PDBParser().get_structure(pdb_name, pdb_path)
            ppb=PPBuilder()
            seq = "" #Re-initializing to prevent any redundancy
            for pp in ppb.build_peptides(structure):
                seq += str(pp.get_sequence())

            #Adding features
            individual_protein_dict = {"PDB File": pdb_name, "Solubility Score": solubility_map[pdb_name] , 
            "Sequence": seq, "Length": len(seq)}

            calculate_SAS(individual_protein_dict, pdb_path)
            calculate_aa_combos(individual_protein_dict,seq)
            calculate_physiochemical_features (individual_protein_dict,seq)
            calculate_residue_features(individual_protein_dict,seq)
            compute_DSSP(individual_protein_dict, pdb_path)

            #Adding the final dictionary with all details to matrix
            pdb_matrix.append(individual_protein_dict)
            
#             #Test code for the first pdb below
#             print(individual_protein_dict)
#             break       
    else:
        print("Pdb list is empty")

def calculate_SAS(temp_dict, pdb_path):
    struct = freesasa.Structure(pdb_path)
    result = freesasa.calc(struct)
    area_classes = freesasa.classifyResults(result, struct)
    polar = area_classes['Polar']
    apolar = area_classes['Apolar']
    sasa_fraction = (polar+apolar)/ len(seq)
    temp_dict.update({"Polar": polar, "Apolar": apolar, "SASA Fraction": sasa_fraction})

def calculate_aa_combos (temp_dict, sequence):
    seq_length = len(sequence)
    
    lys_arg = (sequence.count("K") + sequence.count("R")) /seq_length
    lys_minus_arg = (sequence.count("K") - sequence.count("R")) /seq_length
    
    asp_glu = (sequence.count("D") + sequence.count("E")) /seq_length
    asp_minus_glu = (sequence.count("D") - sequence.count("E")) /seq_length
    
    lys_arg_asp_glu = lys_arg + asp_glu
    lys_arg_asp_minus_glu = lys_arg + asp_minus_glu
    
    phe_tyr_trp = (sequence.count("F")+ sequence.count("Y") + sequence.count("W")) /seq_length
    
    aa_combo_dict = {"Lys+Arg":lys_arg, "Asp+Glu":asp_glu, "Lys+Arg+Asp+Glu":lys_arg_asp_glu,"Phe+Tyr+Trp": phe_tyr_trp,
                     "Lys-Arg":lys_minus_arg, "Asp-Glu":asp_minus_glu, "Lys+Arg+Asp-Glu":lys_arg_asp_minus_glu }
    temp_dict.update(aa_combo_dict)

def calculate_physiochemical_features(temp_dict,sequence):
    analyzed_seq = ProteinAnalysis(sequence)
    
    charge_at_pH7 = analyzed_seq.charge_at_pH(7)
    instability_index = analyzed_seq.instability_index()
    molecular_weight = analyzed_seq.molecular_weight()
    aromaticity = analyzed_seq.aromaticity()
    molar_extinction_coefficient = analyzed_seq.molar_extinction_coefficient()
    gravy = analyzed_seq.gravy() #Grand Average Hyrdopathy - Higher value = More Hydrophobic
    isoelectric_point = analyzed_seq.isoelectric_point()
    helix_fraction, turn_fraction, sheet_fraction = analyzed_seq.secondary_structure_fraction()
    
    physiochem_dict = {"Charge at pH7": charge_at_pH7, "Instability Index": instability_index,"Molecular Wt": molecular_weight, "Aromaticity": aromaticity, "Molar Extinction Coeff": molar_extinction_coefficient,
    "Gravy":gravy, "Isoelectric pt": isoelectric_point, "Helix Fraction": helix_fraction, "Turn Fraction": turn_fraction, 
    "Sheet Fraction": sheet_fraction }
    temp_dict.update(physiochem_dict)
    
    #Adding separately because get_amino_acids_percent() generates a dictionary on its own
    aa_percent = analyzed_seq.get_amino_acids_percent()
    temp_dict.update(aa_percent)

def calculate_residue_features(temp_dict, sequence):
    analyzed_seq = ProteinAnalysis(sequence)
    aa_percent = analyzed_seq.get_amino_acids_percent()
    
    hydrophobicity = 0
    hydrophilicity = 0
    interior__surface_transfer_energy_scale = 0
    surface_fractional_probability = 0
    
    for key in aa_percent.keys():
        hydrophobicity += aa_percent[key]*kd[key]
        hydrophilicity += aa_percent[key]*hw[key]
        surface_fractional_probability += aa_percent[key]*em[key]
        interior__surface_transfer_energy_scale += aa_percent[key]*ja[key]
    
    temp_dict.update({"Hydrophobicity": hydrophobicity,"Hydrophilicity": hydrophilicity, 
                      "Surface Fractional Probability": surface_fractional_probability,
                     "I2S Transfer Energy Scale": interior__surface_transfer_energy_scale})
    temp_dict.update(aa_percent)

def compute_DSSP(temp_dict, pdb_path):
    """compute DSSP and related features"""         
    with open(pdb_path, 'r') as pdb_file:
        try:
            sec_str_based_features = predict.compute_dssp_based(pdb_path)
            temp_dict.update(sec_str_based_features)
        except Exception as e:
            print(pdb_path+' failed to extract secondary structure features')
            print(e)

def pdb_to_csv ():
    csv_columns = list(pdb_matrix[0].keys())
    csv_file = join_p(ROOT_DIR,"training_data.csv")
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in pdb_matrix:
                writer.writerow(data)
    except IOError:
        print("I/O error")

def main():
    generate_pdb_list()
    generate_solubility_mapping()
    pdb_data_curation()
    pdb_to_csv()

if __name__ == '__main__':
    main()
    print("Done :)")